Getting hand landmarks

In [2]:
import mediapipe as mp
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import os

Load Dataset

In [3]:
images_info = []

train_path = "./dataset/"

for class_name in os.listdir(train_path):
    for image_name in os.listdir(train_path + class_name):
        images_info.append({
            'class': class_name,
            'path': './dataset/' + class_name + '/' + image_name
        })
        
print(images_info)

[{'class': '0', 'path': './dataset/0/hand1_0_bot_seg_1_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_bot_seg_2_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_bot_seg_3_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_bot_seg_4_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_bot_seg_5_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_dif_seg_1_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_dif_seg_2_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_dif_seg_3_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_dif_seg_4_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_dif_seg_5_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_left_seg_1_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_left_seg_2_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_left_seg_3_cropped.jpeg'}, {'class': '0', 'path': './dataset/0/hand1_0_left_seg_4_cropped.jpeg'}, {'class': '0', 

In [4]:
model_path = "D:\BINUS\Assignments\Semester 3\AI\SignoAI\hand_landmarker.task"

In [5]:
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    # Loop through the detected hands to visualize
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
        annotated_image,
        hand_landmarks_proto,
        solutions.hands.HAND_CONNECTIONS,
        solutions.drawing_styles.get_default_hand_landmarks_style(),
        solutions.drawing_styles.get_default_hand_connections_style())

        # Get the top left corner of the detected hand's bounding box.
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        print(f"x_coordinates: {x_coordinates[0]}, y_coordinates: {y_coordinates[0]}")
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Draw handedness (left or right hand) on the image.
        cv2.putText(annotated_image, f"{handedness[0].category_name}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image

In [81]:
import itertools

def preprocess_landmark(landmark_list):
    temp_landmark_list = [[0, 0, 0] for _ in range(21)]
    for landmarks in landmark_list:
        for index, landmark in enumerate(landmarks):
            # print(landmark)
            temp_landmark_list[index][0] = landmark.x * 100
            temp_landmark_list[index][1] = landmark.y * 100
            temp_landmark_list[index][2] = landmark.z * 100

    
    # # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))
    print(temp_landmark_list)

    return temp_landmark_list

In [11]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=1,
                                       min_hand_detection_confidence=0.1)
detector = vision.HandLandmarker.create_from_options(options)

In [84]:
import csv
for idx, image_info in enumerate(images_info):
    # STEP 3: Load the input image.
    # image = mp.Image.create_from_file(image_info['path'])
    
    bgr_image = cv2.imread(image_info['path'])
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)
    
    # STEP 4: Detect hand landmarks from the input image.
    detection_result = detector.detect(image)

    # STEP 5: Process the classification result. In this case, visualize it.
    # annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    
    # To show uncomment code below
    # cv2.imshow("Result", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    # STEP 6: Preprocess image landmark to be its relations to each other
    world_landmark = detection_result.hand_world_landmarks
    if(len(world_landmark) == 0):
        # print("No hand detected")
        continue
    
    # print(image_info["class"], ": ",  world_landmark[0][4])
    
    data = preprocess_landmark(world_landmark)
    
    # STEP 7: Append to CSV file
    csv_path = "./model/kp_classifier.csv"
    with open(csv_path, "a") as f:
        writer = csv.writer(f)
        writer.write([image_info["class"], *data])
    
    # if(idx == 5):
    #     break

[-0.14176615643405946, 1.0, 0.7235081294805528, 0.02074038278174236, 0.8936728799416601, 0.5365489653154856, 0.26629024054340994, 0.7349316974942903, 0.5139654756605174, 0.2730763485744283, 0.4972001880997303, 0.35464573694506574, 0.2915484621245296, 0.26205257559726347, 0.2889806960536229, -0.0801714279410907, 0.12967225269071164, 0.1484986965389831, 0.06611731776251986, 0.17453710965301583, 0.03714396878795654, 0.43234658040190604, 0.2512577507273603, 0.1951574368147801, 0.7584267107661408, 0.49212509931599274, 0.3180288484059424, -0.11171171661252453, -0.004266776922907217, 0.08461390142836722, 0.17762517970565897, -0.17459889409809898, -0.1725252524794621, 0.48774424027685476, 0.0029658611380335904, -0.031130924223521955, 0.8387825287264614, 0.17885098989598605, 0.20870662773509974, 0.035495399807125246, -0.13543830350000288, -0.1306753479950787, 0.1740702791546271, -0.2168766997805604, -0.24344578668994946, 0.606635494515823, -0.08240635888304301, -0.09398486022322336, 0.969793280

AttributeError: '_csv.writer' object has no attribute 'write'